# About this kernel 

+ resnet50
+ CurricularFace
+ Mish() activation
+ Ranger (RAdam + Lookahead) optimizer
+ margin = 0.9

## Imports

In [1]:
import sys

sys.path.append('../input/shopee-competition-utils')
sys.path.insert(0,'../input/pytorch-image-models')

In [2]:
import numpy as np 
import pandas as pd 

import torch 
from torch import nn 
from torch.utils.data import Dataset, DataLoader 

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from custom_scheduler import ShopeeScheduler
from custom_activation import replace_activations, Mish
from custom_optimizer import Ranger

import math 
import cv2
import timm 
import os
import random
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import NearestNeighbors
from tqdm.notebook import tqdm 

## Config

In [3]:
class CFG: 
    
    DATA_DIR = '../input/shopee-product-matching/train_images'
    TRAIN_CSV = '../input/shopee-product-matching/train.csv'

    # data augmentation
    IMG_SIZE = 512
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

    SEED = 2021

    # data split
    N_SPLITS = 5
    TEST_FOLD = 0
    VALID_FOLD = 1

    EPOCHS = 8
    BATCH_SIZE = 8

    NUM_WORKERS = 4
    DEVICE = 'cuda:2'

    CLASSES = 6609 
    SCALE = 30
    MARGIN = 0.9

    MODEL_NAME = 'resnet50'
    MODEL_PATH = f'{MODEL_NAME}_curricular_face_epoch_{EPOCHS}_bs_{BATCH_SIZE}_margin_{MARGIN}.pt'
    FC_DIM = 512
    SCHEDULER_PARAMS = {
            "lr_start": 1e-5,
            "lr_max": 1e-5 * 32,
            "lr_min": 1e-6,
            "lr_ramp_ep": 5,
            "lr_sus_ep": 0,
            "lr_decay": 0.8,
        }

## Augmentations

In [4]:
def get_train_transforms():
    return albumentations.Compose(
        [   
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=120, p=0.8),
            albumentations.RandomBrightness(limit=(0.09, 0.6), p=0.5),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0),
        ]
    )

def get_valid_transforms():

    return albumentations.Compose(
        [
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0)
        ]
    )

def get_test_transforms():

    return albumentations.Compose(
        [
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0)
        ]
    )

## Reproducibility

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True # set True to be faster

seed_everything(CFG.SEED)

## Dataset 

In [6]:
class ShopeeDataset(torch.utils.data.Dataset):
    """for training
    """
    def __init__(self,df, transform = None):
        self.df = df 
        self.root_dir = CFG.DATA_DIR
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):

        row = self.df.iloc[idx]

        img_path = os.path.join(self.root_dir,row.image)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = row.label_group

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return {
            'image' : image,
            'label' : torch.tensor(label).long()
        }

In [7]:
class ShopeeImageDataset(torch.utils.data.Dataset):
    """for validating and test
    """
    def __init__(self,df, transform = None):
        self.df = df 
        self.root_dir = CFG.DATA_DIR
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):

        row = self.df.iloc[idx]

        img_path = os.path.join(self.root_dir,row.image)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = row.label_group

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']     
                
        return image,torch.tensor(1)

## Curricular Face + NFNet-L0

In [8]:
'''
credit : https://github.com/HuangYG123/CurricularFace/blob/8b2f47318117995aa05490c05b455b113489917e/head/metrics.py#L70
'''

def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output

class CurricularFace(nn.Module):
    def __init__(self, in_features, out_features, s = 30, m = 0.50):
        super(CurricularFace, self).__init__()

        print('Using Curricular Face')

        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        self.s = s
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.threshold = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        self.kernel = nn.Parameter(torch.Tensor(in_features, out_features))
        self.register_buffer('t', torch.zeros(1))
        nn.init.normal_(self.kernel, std=0.01)

    def forward(self, embbedings, label):
        embbedings = l2_norm(embbedings, axis = 1)
        kernel_norm = l2_norm(self.kernel, axis = 0)
        cos_theta = torch.mm(embbedings, kernel_norm)
        cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
        with torch.no_grad():
            origin_cos = cos_theta.clone()
        target_logit = cos_theta[torch.arange(0, embbedings.size(0)), label].view(-1, 1)

        sin_theta = torch.sqrt(1.0 - torch.pow(target_logit, 2))
        cos_theta_m = target_logit * self.cos_m - sin_theta * self.sin_m #cos(target+margin)
        mask = cos_theta > cos_theta_m
        final_target_logit = torch.where(target_logit > self.threshold, cos_theta_m, target_logit - self.mm)

        hard_example = cos_theta[mask]
        with torch.no_grad():
            self.t = target_logit.mean() * 0.01 + (1 - 0.01) * self.t
        cos_theta[mask] = hard_example * (self.t + hard_example)
        cos_theta.scatter_(1, label.view(-1, 1).long(), final_target_logit)
        output = cos_theta * self.s
        return output, nn.CrossEntropyLoss()(output,label)

In [9]:
class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.CLASSES,
        model_name = CFG.MODEL_NAME,
        fc_dim = CFG.FC_DIM,
        margin = CFG.MARGIN,
        scale = CFG.SCALE,
        use_fc = True,
        pretrained = True):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif 'resnet' in model_name:
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'resnext' in model_name:
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'nfnet' in model_name:
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        if use_fc:
            self.dropout = nn.Dropout(p=0.0)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        self.final = CurricularFace(final_in_features, 
                                           n_classes, 
                                           s=scale, 
                                           m=margin)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        logits = self.final(feature,label)
        return logits

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x


## Engine

In [10]:
def train_fn(model, data_loader, optimizer, scheduler, i):
    model.train()
    fin_loss = 0.0
    tk = tqdm(data_loader, desc = "Epoch" + " [TRAIN] " + str(i+1))

    for t,data in enumerate(tk):
        for k,v in data.items():
            data[k] = v.to(CFG.DEVICE)
        optimizer.zero_grad()
        _, loss = model(**data)
        loss.backward()
        optimizer.step() 
        fin_loss += loss.item() 

        tk.set_postfix({'loss' : '%.6f' %float(fin_loss/(t+1)), 'LR' : optimizer.param_groups[0]['lr']})

    scheduler.step()

    return fin_loss / len(data_loader)

def eval_fn(model, data_loader, i):
    model.eval()
    fin_loss = 0.0
    tk = tqdm(data_loader, desc = "Epoch" + " [VALID] " + str(i+1))

    with torch.no_grad():
        for t,data in enumerate(tk):
            for k,v in data.items():
                data[k] = v.to(CFG.DEVICE)
            _, loss = model(**data)
            fin_loss += loss.item() 

            tk.set_postfix({'loss' : '%.6f' %float(fin_loss/(t+1))})
        return fin_loss / len(data_loader)

In [11]:
def read_dataset():
    df = pd.read_csv(CFG.TRAIN_CSV)
    df['matches'] = df.label_group.map(df.groupby('label_group').posting_id.agg('unique').to_dict())
    df['matches'] = df['matches'].apply(lambda x: ' '.join(x))

    gkf = GroupKFold(n_splits=CFG.N_SPLITS)
    df['fold'] = -1
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=df, groups=df['label_group'])):
        df.loc[valid_idx, 'fold'] = i

    labelencoder= LabelEncoder()
    df['label_group'] = labelencoder.fit_transform(df['label_group'])

    train_df = df[df['fold']!=CFG.TEST_FOLD].reset_index(drop=True)
    train_df = train_df[train_df['fold']!=CFG.VALID_FOLD].reset_index(drop=True)
    valid_df = df[df['fold']==CFG.VALID_FOLD].reset_index(drop=True)
    test_df = df[df['fold']==CFG.TEST_FOLD].reset_index(drop=True)

    train_df['label_group'] = labelencoder.fit_transform(train_df['label_group'])

    return train_df, valid_df, test_df

In [12]:
def precision_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    precision = intersection / len_y_pred
    return precision

def recall_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_true = y_true.apply(lambda x: len(x)).values
    recall = intersection / len_y_true
    return recall

def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [13]:
def get_valid_embeddings(df, model):

    model.eval()

    image_dataset = ShopeeImageDataset(df,transform=get_valid_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.BATCH_SIZE,
        pin_memory=True,
        num_workers = CFG.NUM_WORKERS,
        drop_last=False
    )

    embeds = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.to(CFG.DEVICE)
            label = label.to(CFG.DEVICE)
            feat,_ = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [14]:
def get_neighbors(df, embeddings, KNN = 50, image = True):
    '''
    https://www.kaggle.com/ragnar123/unsupervised-baseline-arcface?scriptVersionId=57121538
    '''

    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    # Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold
    if image:
        thresholds = list(np.arange(0.2,0.4,0.01))
    else:
        thresholds = list(np.arange(0.1, 1, 0.1))
    scores_f1 = []
    scores_recall = []
    scores_precision = []
    for threshold in thresholds:
        predictions = []
        for k in range(embeddings.shape[0]):
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
            posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
            predictions.append(posting_ids)

        df['pred_matches'] = predictions

        df['f1'] = f1_score(df['matches'], df['pred_matches'])
        df['recall'] = recall_score(df['matches'], df['pred_matches'])
        df['precision'] = precision_score(df['matches'], df['pred_matches'])

        score_f1 = df['f1'].mean()
        score_recall = df['recall'].mean()
        score_precision = df['precision'].mean()

        print(f'Our f1 score for threshold {threshold} is {score_f1}, recall score is {score_recall}, mAP score is {score_precision}')

        scores_f1.append(score_f1)
        scores_recall.append(score_recall)
        scores_precision.append(score_precision)

    # create a dataframe to store threshold and scores
    thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores_f1': scores_f1, 'scores_recall': scores_recall, 'scores_precision': scores_precision})

    # obtain best f1 score
    max_score = thresholds_scores[thresholds_scores['scores_f1'] == thresholds_scores['scores_f1'].max()]

    # obtain best threshold and scores
    best_threshold = max_score['thresholds'].values[0]
    best_score_f1 = max_score['scores_f1'].values[0]
    best_score_recall = max_score['scores_recall'].values[0]
    best_score_precision = max_score['scores_precision'].values[0]

    print(f'Our best f1 score is {best_score_f1} and has a threshold {best_threshold}, corresponding recall score is {best_score_recall}, mAP score is {best_score_precision}')

    # Use threshold
    predictions = []
    for k in range(embeddings.shape[0]):
        # Because we are predicting the test set that have 70K images and different label groups, confidence should be smaller
        if image:
            idx = np.where(distances[k,] < best_threshold)[0]
        else:
            idx = np.where(distances[k,] < best_threshold)[0]
        ids = indices[k,idx]
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
        
    df['pred_matches'] = predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    df['recall'] = recall_score(df['matches'], df['pred_matches'])
    df['precision'] = precision_score(df['matches'], df['pred_matches'])
    
    del model, distances, indices
    gc.collect()
    return df, predictions

In [15]:
def get_valid_neighbors(df, embeddings, KNN = 50, threshold = 0.36):

    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)

    predictions = []
    for k in range(embeddings.shape[0]):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
        
    df['pred_matches'] = predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    df['recall'] = recall_score(df['matches'], df['pred_matches'])
    df['precision'] = precision_score(df['matches'], df['pred_matches'])
    
    del model, distances, indices
    gc.collect()
    return df, predictions

# Training 

In [16]:
def run_training():
    train_df, valid_df, test_df = read_dataset()
    train_dataset = ShopeeDataset(train_df, transform = get_train_transforms())

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = CFG.BATCH_SIZE,
        pin_memory = True,
        num_workers = CFG.NUM_WORKERS,
        shuffle = True,
        drop_last = True
    )

    print(train_df['label_group'].nunique())

    model = ShopeeModel()
    model = replace_activations(model, torch.nn.SiLU, Mish())
    model.to(CFG.DEVICE)

    optimizer = Ranger(model.parameters(), lr = CFG.SCHEDULER_PARAMS['lr_start'])
    #optimizer = torch.optim.Adam(model.parameters(), lr = config.SCHEDULER_PARAMS['lr_start'])
    scheduler = ShopeeScheduler(optimizer,**CFG.SCHEDULER_PARAMS)

    best_valid_f1 = 0.
    for i in range(CFG.EPOCHS):
        avg_loss_train = train_fn(model, train_dataloader, optimizer, scheduler, i)

        valid_embeddings = get_valid_embeddings(valid_df, model)
        valid_df, valid_predictions = get_valid_neighbors(valid_df, valid_embeddings)

        valid_f1 = valid_df.f1.mean()
        valid_recall = valid_df.recall.mean()
        valid_precision = valid_df.precision.mean()
        print(f'Valid f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}')

        if valid_f1 > best_valid_f1:
            best_valid_f1 = valid_f1
            print('Valid f1 score improved, model saved')
            torch.save(model.state_dict(),CFG.MODEL_PATH)
        
run_training()

6609
Building Model Backbone for resnet50 model
Using Curricular Face
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.13327893886173361, recall = 0.7967250991740312, precision = 0.08036793692509941
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.13805765180809673, recall = 0.8101540849281752, precision = 0.08412018364087452
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.1598603781596173, recall = 0.813649304270359, precision = 0.10622327367546217
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.19806034238955242, recall = 0.8063681771628802, precision = 0.15664495003023002
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.22552683673051982, recall = 0.8013825632025645, precision = 0.19638082323608583
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.22757580675244476, recall = 0.7814297836260916, precision = 0.21275159010318953
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.2355625775550317, recall = 0.7780427158985455, precision = 0.22670765911571147
Valid f1 score improved, model saved



Our image embeddings shape is (6849, 6609)
Valid f1 score = 0.24027910622442228, recall = 0.774473361645048, precision = 0.2343849218755173
Valid f1 score improved, model saved


In [17]:
def get_test_embeddings(test_df):
    
    model = ShopeeModel()
    model.eval()
    model = replace_activations(model, torch.nn.SiLU, Mish())
    model.load_state_dict(torch.load(CFG.MODEL_PATH))
    model = model.to(CFG.DEVICE)

    image_dataset = ShopeeImageDataset(test_df,transform=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.BATCH_SIZE,
        pin_memory=True,
        num_workers = CFG.NUM_WORKERS,
        drop_last=False
    )

    embeds = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat,_ = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

## Best threshold Search

In [18]:
train_df, valid_df, test_df = read_dataset()

print("Searching best threshold...")
search_space = np.arange(10, 50, 1)

model = ShopeeModel()
model.eval()
model = replace_activations(model, torch.nn.SiLU, Mish())
model.load_state_dict(torch.load(CFG.MODEL_PATH))
model = model.to(CFG.DEVICE)

valid_embeddings = get_valid_embeddings(valid_df, model)

best_f1_valid = 0.
best_threshold = 0.

for i in search_space:
    threshold = i / 100
    valid_df, valid_predictions = get_valid_neighbors(valid_df, valid_embeddings, threshold=threshold)

    valid_f1 = valid_df.f1.mean()
    valid_recall = valid_df.recall.mean()
    valid_precision = valid_df.precision.mean()

    print(f"threshold = {threshold} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")

    if (valid_f1 > best_f1_valid):
        best_f1_valid = valid_f1
        best_threshold = threshold

print("Best threshold =", best_threshold)
print("Best f1 score =", best_f1_valid)
BEST_THRESHOLD = best_threshold

Searching best threshold...
Building Model Backbone for resnet50 model
Using Curricular Face



Our image embeddings shape is (6849, 6609)
threshold = 0.1 -> f1 score = 0.704999033422867, recall = 0.6089248033335279, precision = 0.9888577654832719
threshold = 0.11 -> f1 score = 0.7122974334776271, recall = 0.6202147950545848, precision = 0.984433510860229
threshold = 0.12 -> f1 score = 0.7204147778559296, recall = 0.6335043846371521, precision = 0.9791834286805694
threshold = 0.13 -> f1 score = 0.7283178381320151, recall = 0.6477068029766654, precision = 0.9720157254417917
threshold = 0.14 -> f1 score = 0.7351255829737547, recall = 0.6621183905403236, precision = 0.9629717868400763
threshold = 0.15 -> f1 score = 0.7417772742794345, recall = 0.6777792704097879, precision = 0.9524349130970858
threshold = 0.16 -> f1 score = 0.7448273330778697, recall = 0.6912200792030694, precision = 0.9383612389370954
threshold = 0.17 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
threshold = 0.18 -> f1 score = 0.7437060357383767, recall = 0.7224721367

In [19]:
print("Searching best knn...")

search_space = np.arange(40, 80, 2)

best_f1_valid = 0.
best_knn = 0

for knn in search_space:

    valid_df, valid_predictions = get_valid_neighbors(valid_df, valid_embeddings, KNN=knn, threshold=BEST_THRESHOLD)

    valid_f1 = valid_df.f1.mean()
    valid_recall = valid_df.recall.mean()
    valid_precision = valid_df.precision.mean()

    print(f"knn = {knn} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")

    if (valid_f1 > best_f1_valid):
        best_f1_valid = valid_f1
        best_knn = knn

print("Best knn =", best_knn)
print("Best f1 score =", best_f1_valid)
BEST_KNN = best_knn

Searching best knn...
knn = 40 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 42 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 44 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 46 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 48 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 50 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 52 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 54 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 56 -> f1 score = 0.7468613448929828, recall = 0.707953272659799, precision = 0.9176091876074982
knn = 58 -> f1 score = 0.7468613448929828, recall = 0.707953

In [20]:
test_embeddings = get_valid_embeddings(test_df,model)
test_df, test_predictions = get_valid_neighbors(test_df, test_embeddings, KNN = BEST_KNN, threshold = BEST_THRESHOLD)

test_f1 = test_df.f1.mean()
test_recall = test_df.recall.mean()
test_precision = test_df.precision.mean()
print(f'Test f1 score = {test_f1}, recall = {test_recall}, precision = {test_precision}')


Our image embeddings shape is (6851, 6609)
Test f1 score = 0.7479531504131421, recall = 0.7041759844587446, precision = 0.9222491308172055
